In [352]:
using CSV, DataFrames, Plots, StatPlots;
plotly();

addprocs(3);

In [353]:
iris = CSV.read("iris.csv")

function fixdata(arr)
    result = Array{Float64}(length(arr))
    
    for i = 1:length(arr)
        if arr[i] == "Iris-setosa"
            result[i] = 1.0
        elseif arr[i] == "Iris-versicolor"
            result[i] = 2.0
        else
            result[i] = 3.0
        end
    end
    result
end

iris[:class] = fixdata(iris[:class])
head(iris)

,sepal_length,sepal_width,petal_length,petal_width,class
1,5.1,3.5,1.4,0.2,1.0
2,4.9,3.0,1.4,0.2,1.0
3,4.7,3.2,1.3,0.2,1.0
4,4.6,3.1,1.5,0.2,1.0
5,5.0,3.6,1.4,0.2,1.0
6,5.4,3.9,1.7,0.4,1.0


In [378]:
function dist(x, y)
    sqrt.(sum((x .- transpose(y)).^2, 2))
end

Xy = convert(Array, iris[:, filter(x -> (x in [:petal_length, :sepal_width, :petal_width, :class]), names(iris))])
X = Xy[:, 1:3]
y = Xy[:, 4];

In [381]:
function match_classes(X, C)
    classes = zeros(size(X, 1))
        
    for i = 1:size(X, 1)
        x = X[i, :]
        classes[i] = indmin(dist(C, x))
    end
    classes
end

function k_means(X, k)
    maxes = findmax(X, 1)[1]
    C = convert(SharedArray, rand(k, size(maxes, 1)) .* maxes)

    error = 1

    while error != 0
        C_old = copy(C)
        
        classes = match_classes(X, C)

        @sync @parallel for i = 1:k
            Xs = [X[j, :] for j = 1:size(X, 1) if classes[j] == i]
            if size(Xs, 1) > 0
                C[i, :] = mean(Xs)
            else
                C[i, :] = rand(1, size(maxes, 1)) .* maxes
            end
        end  
        error = sum(sqrt.(sum((C .- C_old).^2, 2)))
    end
    C
end;

In [407]:
C = k_means(X, 3)
classes = match_classes(X, C)
            
scatter(X[:, 1], X[:, 2],X[:, 3], group=classes)
scatter!(C[:,1],C[:,2], C[:,3])

In [408]:
scatter(X[:, 1], X[:, 2],X[:, 3],  group=y)